# 📊 NIFTY50 Stock Market Analysis
This notebook performs complete EDA, trend plotting, and indicator analysis for NIFTY50 stocks (2000–2021).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: interactive plotting
# %matplotlib inline


## 📥 Load and Clean Data

In [ ]:
def load_and_clean_data(filepath):
    df = pd.read_csv(filepath, parse_dates=['Date'])
    df = df[df['Series'] == 'EQ']
    df.sort_values(by=['Symbol', 'Date'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    numeric_cols = ['Prev Close', 'Open', 'High', 'Low', 'Last', 'Close',
                    'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable Volume', '%Deliverble']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    df.dropna(inplace=True)
    return df

df = load_and_clean_data('../data/NIFTY50_all.csv')
df.head()


## 📈 Add Technical Indicators (MA, RSI, Bollinger Bands)

In [ ]:
def moving_averages(df, windows=[50, 200]):
    for window in windows:
        df[f"MA_{window}"] = df.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window).mean())
    return df

def rsi(df, window=14):
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

def bollinger_bands(df, window=20):
    ma = df['Close'].rolling(window).mean()
    std = df['Close'].rolling(window).std()
    df['Bollinger_Upper'] = ma + 2*std
    df['Bollinger_Lower'] = ma - 2*std
    return df

df = moving_averages(df)
df = rsi(df)
df = bollinger_bands(df)


## 📉 Trend and Indicator Visualization

In [ ]:
def plot_stock_trend(df, symbol):
    stock_df = df[df['Symbol'] == symbol]
    plt.figure(figsize=(14, 6))
    plt.plot(stock_df['Date'], stock_df['Close'], label='Close')
    if 'MA_50' in stock_df.columns:
        plt.plot(stock_df['Date'], stock_df['MA_50'], label='50-Day MA')
    if 'MA_200' in stock_df.columns:
        plt.plot(stock_df['Date'], stock_df['MA_200'], label='200-Day MA')
    if 'Bollinger_Upper' in stock_df.columns:
        plt.plot(stock_df['Date'], stock_df['Bollinger_Upper'], linestyle='--', label='Bollinger Upper')
        plt.plot(stock_df['Date'], stock_df['Bollinger_Lower'], linestyle='--', label='Bollinger Lower')
    plt.title(f"{symbol} Price Trend")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_stock_trend(df, "RELIANCE")


## 📊 Volume and Volatility Analysis

In [ ]:
# Top traded stocks
top_volume = df.groupby('Symbol')['Volume'].sum().sort_values(ascending=False).head(10)
print("Top 10 Traded Stocks by Volume:")
print(top_volume)

# Yearly volatility for RELIANCE
df['Returns'] = df.groupby('Symbol')['Close'].pct_change()
df['Year'] = df['Date'].dt.year
reliance_vol = df[df['Symbol'] == 'RELIANCE'].groupby('Year')['Returns'].std()

plt.figure(figsize=(12,5))
sns.barplot(x=reliance_vol.index, y=reliance_vol.values)
plt.title("Yearly Volatility of RELIANCE")
plt.ylabel("Volatility (Standard Deviation)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
